In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input,decode_predictions
import numpy as np
from tensorflow.keras.regularizers import l2
from PIL import Image

Num GPUs Available:  1


In [2]:
#Reading and splitting the images in train,validation and test
animals_path = "C:\\Users\\USUARIO\\animals"
animals_train_path = "C:\\Users\\USUARIO\\animals_train"
animals_test_path = "C:\\Users\\USUARIO\\animals_test"
animals_valid_path = "C:\\Users\\USUARIO\\animals_valid"
subcategories = ["cats","dogs","panda"]
train_ratio = 0.8
for subcategory in subcategories:
    subcategory_folder = os.path.join(animals_path,subcategory)
    image_files = sorted(os.listdir(subcategory_folder))
    num = int(len(image_files) * train_ratio)
    test_num = int(len(image_files) - num)
    train_subfolder = os.path.join(animals_train_path, subcategory)
    test_subfolder = os.path.join(animals_test_path, subcategory)
    valid_subfolder = os.path.join(animals_valid_path,subcategory)
    os.makedirs(train_subfolder, exist_ok=True)
    os.makedirs(test_subfolder, exist_ok=True)
    os.makedirs(valid_subfolder,exist_ok=True)
    for index,i in enumerate(image_files):
        source_path = os.path.join(subcategory_folder,i)
        if index < num:
            if index  < int(num*train_ratio):
                destination_path = os.path.join(train_subfolder,i)
            else:
                destination_path = os.path.join(valid_subfolder,i)
        else:
            destination_path = os.path.join(test_subfolder,i)
        shutil.copyfile(source_path,destination_path)
print("Done")

Done


In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
data_train = train_datagen.flow_from_directory(
    directory=animals_train_path,
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 1920 images belonging to 3 classes.


In [4]:
valid_datagen = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
data_valid= valid_datagen.flow_from_directory(
    directory=animals_valid_path,
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 480 images belonging to 3 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
data_test = test_datagen.flow_from_directory(
    directory=animals_test_path,
    target_size=(224,224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)


Found 600 images belonging to 3 classes.


In [6]:
full_datagen = ImageDataGenerator()
data_full = full_datagen.flow_from_directory(
    directory="C:\\Users\\USUARIO\\animals",
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 3000 images belonging to 3 classes.


In [7]:
#Convolutional Neural Network
def cnn():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=32,activation="relu",kernel_size=(3,3),input_shape=(224,224,3)))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Conv2D(filters=64,activation="relu",kernel_size=(5,5)))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256,activation="relu"))
    model.add(keras.layers.Dense(64,activation="relu"))
    model.add(keras.layers.Dense(3,activation="softmax")) 
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model
def train_test(model):
    model.fit(data_train,validation_data=data_valid,epochs=5)
    test_loss,test_acc = model.evaluate(data_test)
    return test_loss,test_acc
test_loss,test_acc = train_test(cnn())
print(f"Test accuracy was {(test_acc*100):.3f}%")

Epoch 1/5
60/60 [==============================] - 67s 883ms/step - loss: 1.6000 - accuracy: 0.4365 - val_loss: 0.9170 - val_accuracy: 0.5437
Epoch 2/5
60/60 [==============================] - 38s 632ms/step - loss: 0.9190 - accuracy: 0.5547 - val_loss: 0.8468 - val_accuracy: 0.5625
Epoch 3/5
60/60 [==============================] - 36s 603ms/step - loss: 0.8467 - accuracy: 0.5641 - val_loss: 0.7428 - val_accuracy: 0.6062
Epoch 4/5
60/60 [==============================] - 36s 607ms/step - loss: 0.7738 - accuracy: 0.6146 - val_loss: 0.6900 - val_accuracy: 0.6708
Epoch 5/5
19/19 [==============================] - 15s 817ms/step - loss: 0.7546 - accuracy: 0.6283
Test accuracy was 62.833%


In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("0")

In [8]:
#State-of-art convolutional neural network model VGG16 Transfer Learning
model_vgg16 = VGG16(weights="imagenet",include_top=False,input_shape=(224,224,3))
for layer in model_vgg16.layers:
    layer.trainable=False
model_transfer = keras.models.Sequential()
model_transfer.add(model_vgg16)
model_transfer.add(keras.layers.GlobalAveragePooling2D())
model_transfer.add(keras.layers.Dense(64,activation="relu"))
model_transfer.add(keras.layers.Dense(3,activation="softmax"))
model_transfer.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
history  = model_transfer.fit(data_train,validation_data=data_valid,epochs=5)
test_loss,test_acc = model_transfer.evaluate(data_test)
print(f"Test accuracy was {(test_acc*100):.3f}%")

Epoch 1/5
60/60 [==============================] - 49s 683ms/step - loss: 0.9919 - accuracy: 0.5651 - val_loss: 0.8458 - val_accuracy: 0.7229
Epoch 2/5
60/60 [==============================] - 38s 638ms/step - loss: 0.7586 - accuracy: 0.7411 - val_loss: 0.6934 - val_accuracy: 0.7542
Epoch 3/5
60/60 [==============================] - 36s 593ms/step - loss: 0.6013 - accuracy: 0.7880 - val_loss: 0.5743 - val_accuracy: 0.8000
Epoch 4/5
60/60 [==============================] - 36s 599ms/step - loss: 0.5025 - accuracy: 0.8302 - val_loss: 0.5289 - val_accuracy: 0.8083
Epoch 5/5
19/19 [==============================] - 13s 696ms/step - loss: 0.4355 - accuracy: 0.8667
Test accuracy was 86.667%


In [23]:
tf.config.experimental.list_physical_devices('GPU')

[]